In [ ]:
import os
import ansys.speos.script as script
from ansys.speos.core import Speos
import ansys.speos.workflow as workflow

In [ ]:
speos = Speos(host="localhost", port=50077)

In [ ]:
project = script.Project(speos=speos)
opt_material_1 = project.create_optical_property("optical-polished")
opt_material_1.set_volume_optic(index=1.5, absorption=0, constringence=60).set_surface_opticalpolished().commit()

opt_material_2 = project.create_optical_property("black")
opt_material_2.set_volume_none().set_surface_mirror(reflectance=0).commit()


In [ ]:
root_part = project.create_root_part().commit()

source_body = root_part.create_body(name="source_surface").commit()
source_body.create_face(name="face1").set_vertices(
    [-5, -11.228, 5,
     5, -11.228, 5,
     5, -11.228, -5,
     -5, -11.228, -5
     ]).set_facets(
    [1, 2, 0,
     2, 3, 0
     ]).set_normals(
    [0.0, 1.0, 0.0,
     0.0, 1.0, 0.0,
     0.0, 1.0, 0.0,
     0.0, 1.0, 0.0
     ]).commit()

prism_body = root_part.create_body(name="prism_body").commit()
for face_id in range(6):
    face = prism_body.create_face(name="face"+str(face_id)).set_facets(
        [1, 2, 0,
         2, 3, 0
     ])
    if face_id == 0:
        face.set_vertices(
            [-5, 0.0, 5,
             -5, 0.0, -5,
             5, 0.0, -5,
             5, 0.0, 5
             ]).set_normals(
            [-0.0, -1.0, -0.0,
             -0.0, -1.0, -0.0,
             -0.0, -1.0, -0.0,
             -0.0, -1.0, -0.0
             ]).commit()
    elif face_id == 1:
        face.set_vertices(
            [5, 0.0, -5,
             5, 10, -5,
             5, 10, 5,
             5, 0.0, 5,
            ]).set_normals(
            [1.0, -0.0, 0.0,
             1.0, -0.0, 0.0,
             1.0, -0.0, 0.0,
             1.0, -0.0, 0.0
             ]).commit()
    elif face_id == 2:
        face.set_vertices(
            [-5, 10, 5,
             5, 10, 5,
             5, 10, -5,
             -5, 10, -5,
             ]).set_normals(
            [0.0, 1.0, 0.0,
             0.0, 1.0, 0.0,
             0.0, 1.0, 0.0,
             0.0, 1.0, 0.0
             ]).commit()
    elif face_id == 3:
        face.set_vertices(
            [-5, 0.0, -5,
             -5, 10, -5,
             5, 10, -5,
             5, 0.0, -5,
             ]).set_normals(
            [0.0, 0.0, -1.0,
             0.0, 0.0, -1.0,
             0.0, 0.0, -1.0,
             0.0, 0.0, -1.0
             ]).commit()
    elif face_id == 4:
        face.set_vertices(
            [-5,0.0,5,
             -5,10,5,
             -5,10,-5,
             -5,0.0,-5,
             ]).set_normals(
            [-1.0, 0.0, 0.0,
             -1.0, 0.0, 0.0,
             -1.0, 0.0, 0.0,
             -1.0, 0.0, 0.0
             ]).commit()
    elif face_id == 5:
        face.set_vertices(
            [5,0.0,5,
             5,10,5,
             -5,10,5,
             -5,0.0,5,
             ]).set_normals(
            [0.0, 0.0, 1.0,
             0.0, 0.0, 1.0,
             0.0, 0.0, 1.0,
             0.0, 0.0, 1.0
             ]).commit()

opt_material_1.set_geometries(
    geometries=[script.GeoRef.from_native_link(geopath="prism_body")]
).commit()

opt_material_2.set_geometries(
    geometries=[script.GeoRef.from_native_link(geopath="source_surface")]
).commit()

In [ ]:
source_1 = project.create_source(name="surface_source")
source_1.set_spectrum().set_blackbody()
source_1.set_exitance_constant(geometries=[(script.GeoRef.from_native_link(geopath="source_surface/face1"), False)])
source_1.commit()

In [ ]:
sensor_1 = project.create_sensor(name="irradiance_sensor")
sensor_1.set_type_spectral()
sensor_1.set_axis_system(
    [0, 20, 0,
     1, 0, 0,
     0, 0, 1,
     0, 1, 0
     ])
sensor_1.set_illuminance_type_planar(integration_direction=[0, -1, 0])
sensor_1.commit()

In [ ]:
project.preview()

In [ ]:
print(project)

In [ ]:
simu = project.create_simulation(name="direct.1.pyspeos")
simu.set_sensor_paths(["irradiance_sensor"])
simu.set_source_paths(["surface_source"])
simu.set_stop_condition_rays_number(value=20000000)
simu.commit()

In [ ]:
print(project)


In [ ]:
simu.compute_CPU()

In [ ]:
sim2 = project.create_simulation("interactive", feature_type=script.simulation.Interactive)
sim2.set_source_paths(["surface_source"])
sim2.set_sensor_paths(["irradiance_sensor"])
sim2.set_light_expert(True)
sim2.commit()
print(sim2)
result = sim2.compute_CPU()

In [ ]:
print(result[0].path)

from ansys.speos.script.lxp import LightPathFinder
lxp= script.LightPathFinder(speos,result[0].path)
print(lxp.has_sensor_contributions)
lxp.preview(project=project)


In [ ]:
lxp.filter_error_rays()
lxp.preview(project=project,ray_filter=True)

In [ ]:
print(project)

In [ ]:
for ray in lxp.rays:
    print(ray.body_ids)

In [ ]:
project.preview({"opacity": 1})